In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import AxesImage
from matplotlib.animation import FFMpegWriter

%matplotlib inline

In [ ]:
def create_cost_fn(centre_x, centre_y, reg=None, reg_const=0):
    """
    returns a convex cost function
    """
    if reg is None:

        def cost(x, y):
            return (x - centre_x) ** 2 + (y - centre_y) ** 2

    elif reg == "l1":

        def cost(x, y):
            return (
                (x - centre_x) ** 2
                + (y - centre_y) ** 2
                + reg_const * (abs(x) + abs(y))
            )

    elif reg == "l2":

        def cost(x, y):
            return (
                (x - centre_x) ** 2
                + (y - centre_y) ** 2
                + reg_const * (x ** 2 + y ** 2)
            )

    elif reg == "max":

        def cost(x, y):
            return (
                (x - centre_x) ** 2
                + (y - centre_y) ** 2
                + reg_const * (x ** 10 + y ** 10) ** 0.1
            )

    return cost


def create_cost_inputs_outputs(cost, xmin=-10, xmax=10, ymin=-10, ymax=10):
    """
    returns numpy arrays x,y,z where z = cost(x,y)
    """
    x = (
        np.linspace(xmin, xmax, num=100)
        + (np.random.random(100) - 0.5) * xmax / 50
    )
    y = (
        np.linspace(ymin, ymax, num=100)
        + (np.random.random(100) - 0.5) * ymax / 50
    )
    x, y = np.meshgrid(x, y, sparse=False)
    z = cost(x, y)

    return x, y, z


@np.vectorize
def create_y_l1_coordinate(x, r):
    """
    Return positive y coordinate on l1 ball of radius r and centre 0 with x-coordinate x
    """
    if x >= 0:
        return r - x
    elif x <= 0:
        return r + x


@np.vectorize
def create_y_l2_coordinate(x, r):
    """
    Return positive y coordinate on l2 ball of radius r and centre 0 with x-coordinate x
    """
    return sqrt(r ** 2 - x ** 2)


def create_ball_boundary(r, norm):
    """
    returns numpy arrays x and y such that
    [(xi,yi) for i in len(x)] traces out the boundary of
    a ball with radius r using the norm provided
    """
    x = np.linspace(-r, r, 100)

    if norm == "l1":
        y_pos = create_y_l1_coordinate(x, r)
    elif norm == "l2":
        y_pos = np.sqrt(r ** 2 - x ** 2)
    elif norm == "max":
        r10 = r ** 10
        y_pos = (r10 - x ** 10) ** 0.1

    y_neg = -y_pos

    x = np.concatenate([x, x[::-1]])
    y = np.concatenate([y_pos, y_neg[::-1]])

    return x, y


def minimise_cost(cost, x, y):
    """
    given list of x and y coordinates, determine the coordinate which minimises
    the cost function cost(x,y)
    """
    costs = cost(x, y)
    min_index = np.argmin(costs)
    return x[min_index], y[min_index]


def plot_ball(x, y, ax):
    """
    shade in the area enclosed by the coordinates [(xi,yi) for i in len(x)]
    """
    ax.fill(x, y, "b", x, y, "b", alpha=0.2)


def remove_contours(cont):
    """Remove contours from the axis"""
    for c in cont.collections:
        c.remove()

In [ ]:
norm = "l1"

## Without regularsiation, vary cost function, plot contours and ball, plot optimal parameters as red dot

In [ ]:
def initialise(angles, ball_radius=4, norm="l1"):
    centres_x = 5 * np.cos(angles)
    centres_y = 5 * np.sin(angles)

    f, ax = plt.subplots(figsize=(10, 10))

    x_ball, y_ball = create_ball_boundary(4, "l1")
    # comma is needed to pull poly out of list length 1
    ax.fill(x_ball, y_ball, "b", alpha=0.2)

    (marker,) = ax.plot(
        0,
        0,
        marker="o",
        color="red",
        label="Minimum value of cost within shaded region",
    )
    ax.legend()

    def update(i):
        cost = create_cost_fn(centres_x[i], centres_y[i])
        (
            cost_inputs_x,
            cost_inputs_y,
            cost_outputs,
        ) = create_cost_inputs_outputs(cost)
        cont = ax.contour(
            cost_inputs_x,
            cost_inputs_y,
            cost_outputs,
            levels=np.arange(0, 51, 2.5),
        )

        x_min, y_min = minimise_cost(cost, x_ball, y_ball)
        marker.set_xdata(x_min)
        marker.set_ydata(y_min)
        return cont

    return f, ax, update

In [ ]:
angles = np.linspace(0, 2 * np.pi)

f, ax, update = initialise(angles)

ax.set_xlim(-11, 11)
ax.set_ylim(-11, 11)

writer = FFMpegWriter(fps=10)

with writer.saving(f, "l1-cost.mp4", 100):
    for i in range(len(angles)):
        cont = update(i)
        writer.grab_frame()
        # have to clean up contours manually...
        remove_contours(cont)

# don't show the figure in jupyter
plt.close()